# Download weather data for Florence

or any other city

In [51]:
from datetime import datetime

import polars as pl
from meteostat import Hourly, Stations

In [52]:
CITY = "Florence"
COUNTRY = "Italy"

In [53]:
# Load our cities dataset and find the city coordinates
cities = pl.read_parquet("../data/worldcities.parquet")
city_info = cities.filter(city=CITY, country=COUNTRY).row(0, named=True)
city_info


{'city': 'Florence',
 'city_ascii': 'Florence',
 'lat': 43.7714,
 'lng': 11.2542,
 'country': 'Italy',
 'iso2': 'IT',
 'iso3': 'ITA',
 'admin_name': 'Tuscany',
 'capital': 'admin',
 'population': 360930,
 'id': 1380726561}

In [54]:
# Find nearby weather stations
s = Stations()
s = s.nearby(
    lat=city_info["lat"],
    lon=city_info["lng"],
    radius=100000  # in m?
)
stations = s.fetch(10)
stations   # pandas DataFrame

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance
id,,,,,,,,,,,,,,,,
16170,Firenze / Peretola,IT,TC,16170,LIRQ,43.8000,11.2000,40.0,Europe/Rome,1931-01-02,2024-05-16,1976-05-11,2022-04-25,2016-01-01,2022-01-01,5389.250172
16136,Passo Porretta,IT,TC,16136,<NA>,44.0167,11.0000,1313.0,Europe/Rome,NaT,NaT,NaT,NaT,NaT,NaT,34042.297308
LIQD0,Passo Dellaporretta / Porretta Pass / San Momme,IT,TC,<NA>,LIQD,44.0333,10.9333,932.0,Europe/Rome,1990-10-18,1994-08-16,1991-04-03,1994-04-19,NaT,NaT,38846.978891
LIQS0,Siena,IT,TC,<NA>,LIQS,43.3167,11.3500,322.0,Europe/Rome,1944-10-04,1945-01-16,1944-10-05,1945-01-15,1944-01-01,1944-01-01,51146.515151
16164,Volterra,IT,TC,16164,LIQV,43.4000,10.8667,555.0,Europe/Rome,1973-01-01,2017-07-07,NaT,NaT,NaT,NaT,51764.779138
16172,Arezzo,IT,TC,16172,LIQB,43.4667,11.8500,248.0,Europe/Rome,1978-09-14,2024-04-28,1990-03-16,2022-04-03,2016-01-01,2017-01-01,58721.121828
16134,Monte Cimone,IT,ER,16134,LIVC,44.2000,10.7000,2165.0,Europe/Rome,1965-02-01,2024-05-16,1951-01-01,2024-05-10,1951-01-01,2022-01-01,65094.107585
16158,Pisa / S. Giusto,IT,TC,16158,LIRP,43.6833,10.3833,2.0,Europe/Rome,1944-11-24,2024-05-16,1944-11-24,2024-05-10,1945-01-01,2022-01-01,70661.952670
16147,Forli,IT,ER,16147,LIPK,44.2000,12.0667,27.0,Europe/Rome,1973-01-01,2024-05-15,1973-01-24,2012-05-05,NaT,NaT,80603.015417


In [55]:
# Take the first one
station_id = stations.index[0]
station_id

'16170'

In [ ]:
# Get all data
source = Hourly(station_id, start=datetime(1932, 1, 1), end=datetime(2024, 5, 16))
source

In [ ]:
# Create a DataFrame from it
df = pl.DataFrame(source.fetch().reset_index())
df

In [ ]:
# Save
df.write_parquet(f"../data/{CITY.lower()}-meteostat.parquet")